In [2]:
import os
import re
import pandas as pd
import shutil


## Cargar alumnos

Toma los alumnos que estan en alumnos.md y lo carga en un datatable.
El formato del archivo debe ser
```
## Comision N
1. Legajo Apellido, Nombre
```

In [3]:

def cargar_alumos(origen='alumnos.md'):
    # Leer el archivo
    with open(origen, 'r') as file:
        lines = file.readlines()
    
    # Inicializar listas para almacenar los datos
    comisiones = []
    ordenes = []
    legajos = []
    apellidos = []
    nombres = []
    
    # Variables para almacenar la comisión actual
    comision_actual = None
    
    # Procesar cada línea del archivo
    for line in lines:
        line = line.strip()
        if line.startswith('## Comision'):
            # Extraer el número de la comisión
            comision_actual = int(line.split(' ')[-1])
        elif line and line[0].isdigit():
            # Extraer el número de orden, legajo, apellido y nombre
            match = re.match(r'(\d+)\.\s+(\d+)\s+([^,]+),\s+(.+)', line)
            if match:
                comisiones.append(comision_actual)
                ordenes.append(int(match.group(1)))
                legajos.append(match.group(2))
                apellidos.append(match.group(3))
                nombres.append(match.group(4))
    
    # Crear el DataFrame
    data = {
        'comision': comisiones,
        'orden': ordenes,
        'legajo': legajos,
        'apellido': apellidos,
        'nombre': nombres
    }
    df = pd.DataFrame(data)
    
    return df

# Uso de la función


## Detectar datos duplicados

Revisa si los legajos estan duplicados (error), y como curiosidad si hay nombre y apellidos repetidos.

In [4]:
def detectar_duplicados(d):
    # Listar registros con legajo duplicado
    legajos_duplicados = d[d['legajo'].duplicated()]
    print("Registros con legajo duplicado:")
    if len(legajos_duplicados) == 0:
        print("No hay legajos duplicados")
    else:
        print("ERROR: Se encontraron legajos duplicados")
        print(legajos_duplicados)

    # Listar registros con nombre duplicado
    nombres_duplicados = d[d['nombre'].duplicated(keep=False)]
    nombres_duplicados.sort_values(by='nombre', inplace=True)
    num_nombres = len(nombres_duplicados['nombre'].unique())

    print(f"\nHay {num_nombres} Nombres duplicados en {len(nombres_duplicados)} registros")
    print("Registros con nombre duplicado:")
    print(nombres_duplicados)

    # Listar registros con apellido duplicado
    apellidos_duplicados = d[d['apellido'].duplicated(keep=False)]
    apellidos_duplicados.sort_values(by='apellido', inplace=True)

    num_apellidos = len(apellidos_duplicados['apellido'].unique())

    print(f"\nHay {num_apellidos} Apellidos duplicados en {len(apellidos_duplicados)} registros")
    print("Registros con apellido duplicado: ")
    print(apellidos_duplicados)



# Generar trabajo practicos
Copia de la carpeta practivos el tp a cada una de los repositorios 


In [5]:
def ubicar(comision, *camino):
    return '/'.join([f"/Users/adibattista/Documents/GitHub/Lab4-C{comision}", *camino])

def asegurar_directorio(directorio):
    try:
        os.makedirs(directorio)
    except :
        pass 

def crear_directorio(comision, *camino):
    path = ubicar(comision, *camino)
    os.makedirs(path, exist_ok=True)

def copiar_archivos(origen, destino):
    # Verificar si la carpeta de destino existe, si no, crearla
    if not os.path.exists(destino):
        os.makedirs(destino)
    
    # Iterar sobre todos los archivos en la carpeta de origen
    for archivo in os.listdir(origen):
        ruta_origen = os.path.join(origen, archivo)
        ruta_destino = os.path.join(destino, archivo)
        
        # Verificar si es un archivo y no un directorio
        if os.path.isfile(ruta_origen):
            shutil.copy(ruta_origen, ruta_destino)

def publicar(practico):
    alumnos = cargar_alumos()
    for alumno in alumnos.to_dict(orient='records'):
        comision = alumno['comision']
        carpeta = f"{alumno['orden']:02} - {alumno['legajo']} - {alumno['apellido']}, {alumno['nombre']}" 
        destino = ubicar(comision, 'practicos', carpeta, practico)
        copiar_archivos(f'./practicos/{practico}', destino)
    for comision in [3,5,7,9]:
        destino = ubicar(comision)
        shutil.copy(f'./practicos/enunciados/{practico}.md', f"{destino}/README.md")


publicar('tp1')